# Part 2 – Data Analytics

## Step 1: Crawl a real-world dataset

### Data source overview & Data collection
Register in TMDB and obtain an API Key, and comply with the terms of use.   
NOTICE: "This program uses TMDB and the TMDB APIs but is not endorsed, certified, or otherwise approved by TMDB."  
Access rate limit: 40 times/second   


- Dataset：The Movie Database (TMDB) Public API, movie resource entry point: https://www.themoviedb.org/movie
- Call the Discover Movies API: https://api.themoviedb.org/3/discover/movie
- Data type: Movie metadata:
  - The Discover movies API metadata types include: id, title, release_date, vote_average, vote_count, popularity, original_language, genre_ids, overview ect.
  - For more details, please refer to the official documentation:https://developer.themoviedb.org/reference/discover-movie
- Data extraction scale: 200 movie records were extracted page by page, due to the release year


In [73]:
import requests
import pandas as pd


def fetch_tmdb_movies(api_key="7f8c6e28203eb7d6a49fa0caf4466396", start_date="2020-01-01", end_date="2025-12-31", max_results=1000, language="en-UK"):
    """Fetch popular movies from TMDB discover API within a date range."""
    api_key = api_key
    base_url = "https://api.themoviedb.org/3/discover/movie"
    params = {
        "api_key": api_key,
        "language": language,
        "primary_release_date.gte": start_date,
        "primary_release_date.lte": end_date,
        "page": 1,
    }

    records = []
    while len(records) < max_results:
        result = requests.get(base_url, params=params, timeout=10)
        result.raise_for_status()
        data = result.json()
        for movie in data.get("results", []):
            records.append({
                "id": movie.get("id"),
                "title": movie.get("title"),
                "release_date": movie.get("release_date"),
                "vote_average": movie.get("vote_average"),
                "vote_count": movie.get("vote_count"),
                "popularity": movie.get("popularity"),
                "original_language": movie.get("original_language"),
                "genre_ids": movie.get("genre_ids"),
            })
            if len(records) >= max_results:
                break
        if not data.get("total_pages"): 
            break
        elif params["page"] >= data.get("total_pages", 0):
            break
        params["page"] += 1
    movie_data = pd.DataFrame(records)
    return movie_data


### Variables and schema
- `id`: The movie's unique ID
- `title`: Movie title
- `release_date`: Release date
- `vote_average`: TMDB average rating
- `vote_count`: The number of people who voted
- `popularity`: Popularity index
- `original_language`: Original language code
- `genre_ids`: List of type IDs.


In [74]:
df_movies = fetch_tmdb_movies(start_date="2020-01-01", end_date="2025-12-31", max_results=1000)
display(df_movies.head())

,id,title,release_date,vote_average,vote_count,popularity,original_language,genre_ids
0,1062722,Frankenstein,2025-10-17,7.800,1540,485.1768,en,"[18, 27, 14]"
1,1054867,One Battle After Another,2025-09-23,7.524,1478,415.1173,en,"[28, 53, 80]"
2,1248226,Playdate,2025-11-05,6.313,208,309.1327,en,"[28, 35, 10751]"
3,1116465,A Legend,2024-07-05,6.600,79,356.6402,zh,"[28, 12, 14]"
4,1419406,The Shadow's Edge,2025-08-16,6.379,66,487.4401,zh,"[28, 80, 53]"


### Secondary data crawling & Table merging

It's clear that first attempt at scraping data via an API yielded basic information about the films.  
However, based on evaluation, I believe I need more detailed data for each film to improve analysis:
 - `runtime`: Video length 
 - `budget`: Film cost/budget 
 - `revenue`: Box office revenue 
 - `status`: Release status 
 - `original_title`: Original language title 
 - `tagline`: Marketing tagline 
 - `spoken_languages`: Spoken languages list 
 - `production_countries`: Production countries 

I merge the two tables using the movie's unique ID.


In [75]:
import time

def fetch_movie_detail(movie_id, api_key="7f8c6e28203eb7d6a49fa0caf4466396"):
    url = f"https://api.themoviedb.org/3/movie/{movie_id}"
    resp = requests.get(url, params={"api_key": api_key}, timeout=10)
    resp.raise_for_status()
    details = resp.json()

    spoken_lang_names = ", ".join(
        sorted(
            {
                lang.get("english_name") or lang.get("name")
                for lang in (details.get("spoken_languages") or [])
                if lang.get("english_name") or lang.get("name")
            }
        )
    )
    production_country_codes = ", ".join(
        sorted(
            {
                country.get("iso_3166_1") or country.get("name")
                for country in (details.get("production_countries") or [])
                if country.get("iso_3166_1") or country.get("name")
            }
        )
    )

    return {
        "id": movie_id,
        "runtime": details.get("runtime"),
        "budget": details.get("budget"),
        "revenue": details.get("revenue"),
        "status": details.get("status"),
        "original_title": details.get("original_title"),
        "tagline": details.get("tagline"),
        "spoken_languages": spoken_lang_names,
        "production_countries": production_country_codes,
    }

detail_record = []
for movie_id in df_movies["id"]:
    detail_record.append(fetch_movie_detail(movie_id))
    time.sleep(0.1)  # add a sleep time to control access frequency

df_movie_details = pd.DataFrame(detail_record)

display(df_movie_details.head())

"""merge the two tables with unique ID"""
df_movie_full = df_movies.merge(df_movie_details, on = "id", how = "left")
display(df_movie_full.head())


,id,runtime,budget,revenue,status,original_title,tagline,spoken_languages,production_countries
0,1062722,150,120000000,144496,Released,Frankenstein,Only monsters play God.,"Danish, English, French","CA, US"
1,1054867,162,130000000,200300000,Released,One Battle After Another,"Some search for battle, others are born into i...","English, Spanish, Telugu",US
2,1248226,95,0,0,Released,Playdate,Playtime just got real.,English,"CA, US"
3,1116465,129,0,11242056,Released,传说,,"English, Mandarin","CN, HK"
4,1419406,142,0,702538,Released,捕风追影,,Mandarin,"CN, HK"


,id,title,release_date,vote_average,vote_count,popularity,original_language,genre_ids,runtime,budget,revenue,status,original_title,tagline,spoken_languages,production_countries
0,1062722,Frankenstein,2025-10-17,7.800,1540,485.1768,en,"[18, 27, 14]",150,120000000,144496,Released,Frankenstein,Only monsters play God.,"Danish, English, French","CA, US"
1,1054867,One Battle After Another,2025-09-23,7.524,1478,415.1173,en,"[28, 53, 80]",162,130000000,200300000,Released,One Battle After Another,"Some search for battle, others are born into i...","English, Spanish, Telugu",US
2,1248226,Playdate,2025-11-05,6.313,208,309.1327,en,"[28, 35, 10751]",95,0,0,Released,Playdate,Playtime just got real.,English,"CA, US"
3,1116465,A Legend,2024-07-05,6.600,79,356.6402,zh,"[28, 12, 14]",129,0,11242056,Released,传说,,"English, Mandarin","CN, HK"
4,1419406,The Shadow's Edge,2025-08-16,6.379,66,487.4401,zh,"[28, 80, 53]",142,0,702538,Released,捕风追影,,Mandarin,"CN, HK"


### Storage as CSV
- Storage path：local



In [76]:
from pathlib import Path

output_path = Path("C:\\Users\\10525\\Desktop\\SDPA-final\\SDPA_EMATM0048_2720314\\data\\tmdb_movies_2020-2025.csv")
output_path.parent.mkdir(parents=True,exist_ok=True)
df_movie_full.to_csv(output_path)


## Step 2: Perform data preparation and cleaning 

### Load data
- This explains the parameters for using `pandas.read_csv` to read CSV files.
- Record the initial DataFrame shape and data type checks.

In [83]:
movie_data = pd.read_csv("C:\\Users\\10525\\Desktop\\SDPA-final\\SDPA_EMATM0048_2720314\\data\\tmdb_movies_2020-2025.csv")
movie_data.drop(columns=["Unnamed: 0"], inplace = True)


In [84]:
data_shape = movie_data.shape
print(f"initial data shape: {data_shape[0]} rows, {data_shape[1]} columns")

initial data shape: 1000 rows, 16 columns


In [85]:
movie_data.dtypes

id                        int64
title                    object
release_date             object
vote_average            float64
vote_count                int64
popularity              float64
original_language        object
genre_ids                object
runtime                   int64
budget                    int64
revenue                   int64
status                   object
original_title           object
tagline                  object
spoken_languages         object
production_countries     object
dtype: object

In [88]:
movie_data.describe()

,id,vote_average,vote_count,popularity,runtime,budget,revenue
count,1.000000e+03,1000.000000,1000.000000,1000.000000,1000.000000,1.000000e+03,1.000000e+03
mean,9.871824e+05,6.016325,925.145000,18.747516,101.241000,2.374054e+07,6.511449e+07
std,3.009462e+05,1.872346,2106.174887,38.193507,29.019598,5.883557e+07,2.136382e+08
min,1.349400e+04,0.000000,0.000000,4.829800,0.000000,0.000000e+00,0.000000e+00
25%,7.602728e+05,5.596750,10.000000,8.401225,90.000000,0.000000e+00,0.000000e+00
50%,1.008841e+06,6.400000,68.000000,9.916000,100.000000,0.000000e+00,0.000000e+00
75%,1.235927e+06,7.085500,611.000000,14.014900,114.000000,1.226500e+07,7.458928e+06
max,1.580902e+06,10.000000,21280.000000,487.440100,242.000000,5.839000e+08,2.330225e+09


In [87]:
movie_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    1000 non-null   int64  
 1   title                 1000 non-null   object 
 2   release_date          1000 non-null   object 
 3   vote_average          1000 non-null   float64
 4   vote_count            1000 non-null   int64  
 5   popularity            1000 non-null   float64
 6   original_language     1000 non-null   object 
 7   genre_ids             1000 non-null   object 
 8   runtime               1000 non-null   int64  
 9   budget                1000 non-null   int64  
 10  revenue               1000 non-null   int64  
 11  status                1000 non-null   object 
 12  original_title        1000 non-null   object 
 13  tagline               601 non-null    object 
 14  spoken_languages      984 non-null    object 
 15  production_countries  

The extracted data contains null values, and some data is unreasonable, for example:  
 - The minimum value of `revenue`, `runtime`, and `budget` is 0, which is inconsistent with reality.  
 - `tagline`, `spoken_languages`, and `production_countries` contain empty values.  

Therefore, "Missing values & Outliers" will be to preprocess the data to make it as compatible with our requirements as possible.

### Missing values & Outliers
- Describe the methods and strategies for detecting missing values ​​(deletion/filling/interpolation), and explain the reasons for choosing them.
- Describe the rules or statistical methods for detecting outliers/inconsistent data, and the corrective/filtering measures and their basis.